In [1]:
import pandas as pd
import numpy as np
from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Embedding

Using TensorFlow backend.


In [2]:
df1 = pd.read_csv('./Datasets_final/q1/train.csv')
df2 = pd.read_csv('./Datasets_final/q1/test.csv')

In [3]:
docs_t = df1['text'].values
labels = df1['labels'].values

x_val_t = df2['text'].values

In [4]:
docs = []
x_val = []

In [5]:
for i in docs_t:
    docs.append(i.replace('#', ' '))
    
for i in x_val_t:
    x_val.append(i.replace('#', ' '))

In [6]:
docs = np.array(docs)
x_val = np.array(x_val)

## prepare tokenizer

In [7]:
t = Tokenizer()
t.fit_on_texts(docs)
vocab_size = len(t.word_index) + 1

## integer encode the documents

In [8]:
encoded_docs = t.texts_to_sequences(docs)
# print(encoded_docs)

## pad documents to a max length of 4 words

In [9]:
max_length = 20
padded_docs = pad_sequences(encoded_docs, maxlen=max_length, padding='post')
# print(padded_docs)

## load the whole embedding into memory

In [10]:
embeddings_index = dict()
f = open('./Datasets_final/q1/glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 400000 word vectors.


## create a weight matrix for words in training docs

In [11]:
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

## define model

In [12]:
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=20, trainable=True)
model.add(e)
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

## compile the model

In [13]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

## summarize the model

In [14]:
print(model.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 100)           2021600   
_________________________________________________________________
flatten_1 (Flatten)          (None, 2000)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 2001      
Total params: 2,023,601
Trainable params: 2,023,601
Non-trainable params: 0
_________________________________________________________________
None


## fit the model

In [15]:
model.fit(padded_docs, labels, epochs=500, verbose=0)

/usr/local/lib/python3.7/site-packages/tensorflow_core/python/framework/indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


## evaluate the model

In [16]:
loss, accuracy = model.evaluate(padded_docs, labels, verbose=0)
print('Accuracy: %f' % (accuracy*100))

Accuracy: 99.677175


In [17]:
encoded_docs_val = t.texts_to_sequences(x_val)
# print(encoded_docs_val)

In [18]:
max_length = 20
padded_docs_val = pad_sequences(encoded_docs_val, maxlen=max_length, padding='post')
# print(padded_docs_val)

In [19]:
y_pred = model.predict(padded_docs_val)

In [20]:
y_pred1 = []
for i in y_pred:
    y_pred1.append(int(round(i[0])))

In [21]:
y_pred1 = np.array(y_pred1)

In [22]:
pd.DataFrame(y_pred1, columns=['labels']).to_csv("./Datasets_final/q1/final_submission.csv", index=True)